# 1. Load pretrained model

In [1]:
import torch
import torch.nn as nn
from torchvision.models.vgg import vgg11, vgg13, vgg16, vgg19 
from torch.ao.quantization import QuantStub, DeQuantStub

weight_path = "best_model_vgg11.pt"


model = vgg11()
# model.to(torch.device('cpu'))
model.load_state_dict(torch.load(weight_path, map_location="cpu"))

model.eval()

return_nodes = {
    "features.1": "layer1",
    "features.4": "layer2",
    "features.7": "layer3",
    "features.9": "layer4",
    "features.12": "layer5",
    "features.14": "layer6",
    "features.17": "layer7",
    "features.19": "layer8",
}


/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. select primary layers

In [2]:
from pca import extract_primary_layers

x = torch.rand(1, 3, 224, 224)
primary_layers = extract_primary_layers(x, model, return_nodes, threshold = 0.99, num_layers = 3, status_print = False)
primary_layers

[('features.19', 0.01953125),
 ('features.14', 0.109375),
 ('features.17', 0.123046875)]

# 3. quantize